# Face Greeter: Real-Time Video Analytics System

This Jupyter Notebook powers the backend for FaceGreeter, a real-time video analytics system for businesses. This simplified version tests Flask and ngrok stability to debug kernel crashes and ngrok tunnel issues. The full system uses `face_recognition` for face detection, OpenCV for video processing, and a Flask API to manage camera connections, video streaming, and customer insights via the xAI Grok API. It integrates with the Vercel frontend at https://facegreeter.vercel.app.

## Prerequisites
- Google Colab environment with internet access.
- `NGROK_AUTHTOKEN` stored in Colab Secrets.
- GitHub repository `ai-greeter-core` at https://github.com/ATEMMETA/ai-greeter-core.

## Steps
1. Clone the ai-greeter-core repository.
2. Install dependencies from requirements.txt.
3. Set up environment variables.
4. Run simplified Flask API with ngrok.
5. Test stability and monitor resources.

**Security Note**: Securely handle `NGROK_AUTHTOKEN` in Colab Secrets.

In [ ]:
# Clone repository and install dependencies
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'  # Disable frozen modules warning

# Clone ai-greeter-core repository
!git clone https://github.com/ATEMMETA/ai-greeter-core.git /content/ai-greeter-core

# Purge pip cache
!pip cache purge

# Install Python dependencies from requirements.txt
!pip install -r /content/ai-greeter-core/requirements.txt

# Verify installed packages
!pip list | grep -E 'flask|pyngrok|python-dotenv'
print("Dependencies installed. Please go to 'Runtime' > 'Restart runtime' and then re-run all cells.")
print("Monitor RAM/CPU usage in the top-right corner of Colab during the next steps.")

In [ ]:
# Set up environment variables
from google.colab import userdata
import os

# Check for required secrets
ngrok_token = userdata.get("NGROK_AUTHTOKEN")
if not ngrok_token:
    raise ValueError("NGROK_AUTHTOKEN not found in Colab Secrets.")

# Write .env file
with open('/content/ai-greeter-core/.env', 'w') as f:
    f.write(f'NGROK_AUTHTOKEN={ngrok_token}\n')
    f.write('VERCEL_URL=https://facegreeter.vercel.app\n')
    f.write('VERCEL_API_BASE_URL=https://facegreeter.vercel.app\n')

In [ ]:
# Run Flask with ngrok
from pyngrok import ngrok
import os
import subprocess
import time
from dotenv import load_dotenv

# Kill existing ngrok and Flask processes
os.system('pkill ngrok')
os.system('pkill python')
os.system('fuser -k 8000/tcp')  # Kill any process on port 8000

# Load environment variables
load_dotenv('/content/ai-greeter-core/.env')
ngrok_token = os.getenv("NGROK_AUTHTOKEN")
if not ngrok_token:
    raise ValueError("NGROK_AUTHTOKEN not found in .env file.")

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Start Flask server with error logging
print("Starting Flask server...")
flask_process = subprocess.Popen(
    ['python', '/content/ai-greeter-core/combined_app.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for Flask to start
time.sleep(5)

# Check if Flask is running
if flask_process.poll() is not None:
    stdout, stderr = flask_process.communicate()
    print(f"Flask failed to start. Error: {stderr}")
    raise RuntimeError("Flask server crashed. Check error above.")

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f"Ngrok failed to start: {str(e)}")
    flask_process.terminate()
    raise

print(f'Flask server started with PID: {flask_process.pid}')

# Debug: Check running processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Keep the notebook running
try:
    input('Press Enter to stop the Flask server...')
finally:
    flask_process.terminate()
    os.system('pkill ngrok')
    os.system('pkill python')
    os.system('fuser -k 8000/tcp')